In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
import random

### Funciones Auxiliares

In [ ]:
def leer_archivo(filename):
    '''
    Recibe el nombre de un archivo de texto con la descripción de un grafo en formato TSPLIB y devuelve los paranetros necesarios para armar un grafo de NetworkX.
    '''
    
    with open(filename, 'r') as file:
        lines = file.readlines()
    
    dimension = 0
    pesos_aristas = []
    reading_weights = False
    
    for line in lines:
        if line.startswith("DIMENSION"):
            dimension = int(line.split()[-1])
        elif line.startswith("EDGE_WEIGHT_SECTION"):
            reading_weights = True
            continue
        elif line.startswith("EOF"):
            break
        elif reading_weights:
            pesos_aristas.extend(map(int, line.split()))
            
    return dimension, pesos_aristas

In [ ]:
def crear_grafo_ATSP(dimension, pesos_aristas):
    '''
    Recibe la dimensión de un grafo y una lista de pesos de aristas y devuelve un grafo de NetworkX.
    '''
    
    G = nx.DiGraph()

    index = 0
    for i in range(dimension):
        for j in range(dimension):
            if i != j:
                peso = pesos_aristas[index]
                G.add_edge(i, j, weight=peso)
            index += 1

    return G

In [ ]:
def calcular_costo(G, solucion):
    '''
    Recibe un grafo de NetworkX y una solución al problema ATSP y devuelve el costo total de la solución, sumando los pesos de las aristas recorridas.
    '''
    
    total_cost = 0
    
    for i in range(len(solucion) - 1):
        total_cost += G[solucion[i]][solucion[i+1]]['weight']
        
    total_cost += G[solucion[-1]][solucion[0]]['weight']
    
    return total_cost

In [66]:
def distancia_promedio_de_vecinos(G, nodo, visited):
    '''
    Recibe un grafo de NetworkX, un nodo, y una lista de nodos visitados, y devuelve la distancia promedio de los vecinos del nodo que aun no han sido visitados.
    
    Observación: Se restan los vecinos visitados ya que no tiene sentido contemplarlos en la distancia promedio, ya que ya han sido visitados. 
                 Este cambio mejora la complejidad computacional del algoritmo. 
    '''
    
    vecinos = set(G.neighbors(nodo)) - set(visited) # O(n)
    
    if len(vecinos) == 0: # O(1)
        return 0
    else:
        total_distance = sum(nx.shortest_path_length(G, nodo, neighbor) for neighbor in vecinos) # O(n^3)
        return total_distance / len(vecinos) # O(1)
    
# Complejidad total: O(n^3)

In [ ]:
def visualizar_grafo(G):
    pos = nx.spring_layout(G)
    edge_labels = {(u, v): data['weight'] for u, v, data in G.edges(data=True)}
    
    plt.figure(figsize=(10, 10))
    
    nx.draw(G, pos, with_labels=True, node_size=700, node_color='skyblue', font_size=10, font_weight='bold', arrows=True)
    nx.draw_networkx_edge_labels(G, pos, edge_labels=edge_labels, font_color='red')
    
    plt.title("Graph Visualization of ATSP")
    plt.show()

In [ ]:
def visualize_hamiltonian_circuit(G, circuit):
    pos = circular_layout_based_on_circuit(circuit)
    plt.figure(figsize=(5, 5))
    
    # Draw nodes
    nx.draw_networkx_nodes(G, pos, nodelist=circuit, node_size=700, node_color='skyblue')

    # Draw only the edges in the circuit
    edges = [(circuit[i], circuit[i + 1]) for i in range(len(circuit) - 1)]
    edges.append((circuit[-1], circuit[0]))  # Close the circuit
    
    edge_labels = {(u, v): G[u][v]['weight'] for u, v in edges}
    
    
    nx.draw_networkx_edges(G, pos, edgelist=edges, edge_color='blue', width=2, arrows=True)
    nx.draw_networkx_edge_labels(G, pos, edge_labels=edge_labels, font_color='red')
    
    # Draw node labels
    nx.draw_networkx_labels(G, pos, font_size=10, font_weight='bold')
    
    plt.title("Hamiltonian Circuit Visualization")
    plt.show()

def circular_layout_based_on_circuit(circuit):
    num_nodes = len(circuit)
    angle_step = 2 * np.pi / num_nodes
    pos = {}
    for i, node in enumerate(circuit):
        angle = i * angle_step
        pos[node] = np.array([np.cos(angle), np.sin(angle)])
    return pos

### Heurísticas

In [ ]:
def heuristica_1(G):
    '''
    Recibe un grafo de NetworkX y devuelve una solución al problema ATSP utilizando el método: agregar a la solución al vecino mas cercano.
    '''
    
    # n = |G|
    
    # Inicializo la solución
    start_node = 0 #O(1)
    visited = [start_node] #O(1)
    current_node = start_node #O(1)

    # Mientras no haya visitado todos los nodos
    while len(visited) < len(G.nodes): #O(n)
        next_node = min(
            (node for node in G.neighbors(current_node) if node not in visited),
            key=lambda node: G[current_node][node]['weight'],
            default=None
        ) #O(n), como es completo, entonces en cada iteración se recorren todos los nodos
        if next_node is None: #O(1)
            break
        
        visited.append(next_node) #O(1)
        current_node = next_node #O(1)
    
    # Calculo el costo totan de la solución
    total_cost = calcular_costo(G, visited) #O(n) recorre al menos una vez cada nodo
    
    return visited, total_cost

# Complejidad total: O(n^2)

In [ ]:
def heuristica_2(G):
    '''
    Recibe un grafo de NetworkX y devuelve una solución al problema ATSP utilizando el metodo: agregar a la solución al vecino con menor promedio a sus vecinos.
    '''
    
    # n = |G|
    
    # Inicializo la solución
    start_node = 0 #O(1)
    visited = [start_node]
    current_node = 0
    
    # Mientras no haya visitado todos los nodos
    while len(visited) < len(G.nodes): #O(n)
        neighbors = set(G.neighbors(current_node)) - set(visited) #O(n), porque tengo que recorrer un set de n elementos
        if not neighbors: #O(1) 
            break
        
        best_average = float('inf') #O(1)
        next_node = None #O(1) 
        
        for node in neighbors: #O(n)
            if node not in visited: #O(1), porque es una lista
                average = distancia_promedio_de_vecinos(G, node, visited) #O(n^3)
                if average < best_average: #O(1)
                    best_average = average #O(1)
                    next_node = node #O(1) 
        
        visited.append(next_node) #O(1)
        current_node = next_node #O(1) 
    
    # Calculo el costo total de la solución
    total_cost = calcular_costo(G, visited) #O(n)
    
    return visited,total_cost

# Complejidad total: O(n^5)

In [ ]:
def heuristica_3(G):
    '''
    Recibe un grafo de NetworkX y devuelve una solución al problema ATSP utilizando el metodo: agregar a la solución al vecino con menor distancia inverso.
    '''
    
    # n = |G|
    
    start_node = 0 #O(1)
    visited = [start_node] #O(1)
    current_node = start_node #O(1)

    while len(visited) < len(G.nodes): #O(n)
        next_node = min( 
            (node for node in G.neighbors(current_node) if node not in visited),
            key=lambda node: G[node][current_node]['weight'],
            default=None
        ) #O(n), porque recorre todos los nodos 
        if next_node is None: #O(1)
            break
        
        visited.append(next_node) #O(1)
        current_node = next_node #O(1) 
    
    total_cost = calcular_costo(G, visited) #O(n)
    
    return visited, total_cost

# Complejidad total: O(n^2)

### Operadores

In [ ]:
def operador_swap(solucion):
    
    # n = |solucion|

    neighborhood = [] #O(1)

    # Recorro todas los pares de nodos (i,j) donde i != j
    for i in range(len(solucion)): #O(n)
        for j in range(i + 1, len(solucion)): #O(n)
            # Swapeo los nodos en la posicion i y j
            nueva_solucion = solucion[:] #O(n)
            nueva_solucion[i], nueva_solucion[j] = nueva_solucion[j], nueva_solucion[i] #O(1)
            neighborhood.append(nueva_solucion) #O(1)

    return neighborhood #O(1)

# Complejidad total: O(n^3)
# Tamaño del neighborhood : O(n^2) -> este calculo viene de la combinatoria (n | 2), de n elementos tomo 2, por lo que tendría n! / (n-2)!2! = n(n-1)/2 ≈ O(n^2)

In [ ]:
def operador_insert(solucion):
    
    # n = |solucion|

    neighborhood = [] #O(1)

    # Iterate over all nodes in the circuit
    for i in range(len(solucion)): #O(n)
        # Remove node from its current position
        nodo = solucion[i] #O(1)
        solucion_temp = solucion[:i] + solucion[i+1:] #O(n)

        # Insert node in every possible position in the reduced circuit
        for j in range(len(solucion_temp) + 1): #O(n)
            nueva_solucion = solucion_temp[:j] + [nodo] + solucion_temp[j:] #O(n)
            
            neighborhood.append(nueva_solucion) #O(1)

    return neighborhood #O(1)

# Complejidad total: O(n^3)
# Tamaño del neighborhood : O(n^2) -> Cada nodo puede estar en n posiciones diferentes, por lo que el tamaño del neighborhood es n^2

In [1]:
def operador_2opt(solucion):
    
    neighborhood = [] #O(1)
    
    for i in range(1, len(solucion) - 2): #O(n)
        for k in range(i + 1, len(solucion) - 1): #O(n) 
            new_solution = solucion[:i] + solucion[i:k+1][::-1] + solucion[k+1:] #O(n)
            neighborhood.append(new_solution) #O(1)
    
    return neighborhood #O(1)

# Complejidad total: O(n^3)
# Tamaño del neighborhood : O(n^2) -> El tamaño de la vecindad es el número de pares (i,k) donde 𝑖 < 𝑘, entonces tenemos la combinatoria (n | 2) ≈ O(n^2)

### Búsqueda local

In [ ]:
def busqueda_local(G, solucion, operador:str):

        print(f'Iniciando busqueda local con operador {operador}')
        
        best_solution = solucion
        best_cost = calcular_costo(G, best_solution)

        i:int = 0
        
        while i < 1000:
            
            if operador == "swap":
                neighborhood = operador_swap(best_solution) 
            elif operador == "insert":
                neighborhood = operador_insert(best_solution)
            elif operador == "2opt":
                neighborhood = operador_2opt(best_solution)
                
            improved = False

            if operador == "all":
                
                neighborhood_2opt = operador_2opt(best_solution)
                for neighbor in neighborhood_2opt:
                    cost = calcular_costo(G, neighbor)
                    if cost < best_cost:
                        best_solution = neighbor
                        best_cost = cost
                        improved = True           
                
                neighborhood_swap = operador_swap(best_solution)
                for neighbor in neighborhood_swap:
                    cost = calcular_costo(G, neighbor)
                    if cost < best_cost:
                        best_solution = neighbor
                        best_cost = cost
                        improved = True
                
                neighborhood_insert = operador_insert(best_solution)
                for neighbor in neighborhood_insert:
                    cost = calcular_costo(G, neighbor)
                    if cost < best_cost:
                        best_solution = neighbor
                        best_cost = cost
                        improved = True            
                
            
            else:
                for neighbor in neighborhood:
                    cost = calcular_costo(G, neighbor)
                    
                    if cost < best_cost:
                        best_solution = neighbor
                        best_cost = cost
                        improved = True
    
            i+=1
            if not improved:
                break
            
        return best_solution, best_cost

### Cargado de instancia

In [ ]:
filename = './instancias/ftv55.atsp'
dimension, pesos_aristas = leer_archivo(filename)

G = crear_grafo_ATSP(dimension, pesos_aristas)

#### Calculo de costo con heurísticas

In [ ]:
CH_heuristica1,total_cost = heuristica_1(G)

print("Costo total 1: ", total_cost)

In [ ]:
CH_heuristica2,total_cost2 = heuristica_2(G)

print("Costo total 2: ", total_cost2)  

In [ ]:
CH_heuristica3,total_cost3 = heuristica_3(G)

print("Costo total 3: ", total_cost3)  

#### Mejora de soluciones con búsqueda local

In [ ]:
# Heuristica 1

print("Solucion inicial: ", CH_heuristica1)
print("Costo incial: ", total_cost)

solucion_mejorada_swap = busqueda_local(G, CH_heuristica1, "swap")

print("Solucion mejorada con swap: ", solucion_mejorada_swap[0])
print("Costo total: ", solucion_mejorada_swap[1])

solucion_mejorada_insert = busqueda_local(G, CH_heuristica1, "insert")

print("Solucion mejorada con insert: ", solucion_mejorada_insert[0])
print("Costo total: ", solucion_mejorada_insert[1])

solucion_mejorada_2opt = busqueda_local(G, CH_heuristica1, "2opt")

print("Solucion mejorada con 2opt: ", solucion_mejorada_2opt[0])
print("Costo total: ", solucion_mejorada_2opt[1])

In [ ]:
# Heuristica 2

print("Solucion inicial: ", CH_heuristica2)
print("Costo incial: ", total_cost2)

solucion_mejorada_insert = busqueda_local(G, CH_heuristica2, "insert")

print("Solucion mejorada con insert: ", solucion_mejorada_insert[0])
print("Costo total: ", solucion_mejorada_insert[1])

solucion_mejorada_swap = busqueda_local(G, CH_heuristica2, "swap")

print("Solucion mejorada con swap: ", solucion_mejorada_swap[0])
print("Costo total: ", solucion_mejorada_swap[1])

solucion_mejorada_2opt = busqueda_local(G, CH_heuristica2, "2opt")

print("Solucion mejorada con 2opt: ", solucion_mejorada_2opt[0])
print("Costo total: ", solucion_mejorada_2opt[1])

In [ ]:
# Heuristica 3

print("Solucion inicial: ", CH_heuristica3)
print("Costo incial: ", total_cost3)

solucion_mejorada_insert = busqueda_local(G, CH_heuristica3, "insert")

print("Solucion mejorada con insert: ", solucion_mejorada_insert[0])
print("Costo total: ", solucion_mejorada_insert[1])

solucion_mejorada_swap = busqueda_local(G, CH_heuristica3, "swap")

print("Solucion mejorada con swap: ", solucion_mejorada_swap[0])
print("Costo total: ", solucion_mejorada_swap[1])

solucion_mejorada_2opt = busqueda_local(G, CH_heuristica3, "2opt")

print("Solucion mejorada con 2opt: ", solucion_mejorada_2opt[0])
print("Costo total: ", solucion_mejorada_2opt[1])

#### Mejora de soluciones con los operadores en conjunto

In [ ]:
# Heuristica 1

print("Solucion inicial: ", CH_heuristica1)
print("Costo incial: ", total_cost)

solucion_mejorada_all = busqueda_local(G, CH_heuristica1, "all")

print("Solucion mejorada con todos: ", solucion_mejorada_all[0])
print("Costo total: ", solucion_mejorada_all[1])

In [ ]:
# Heuristica 2

print("Solucion inicial: ", CH_heuristica2)
print("Costo incial: ", total_cost2)

solucion_mejorada_all2 = busqueda_local(G, CH_heuristica2, "all")

print("Solucion mejorada con todos: ", solucion_mejorada_all2[0])
print("Costo total: ", solucion_mejorada_all2[1])

In [ ]:
# Heuristica 3

print("Solucion inicial: ", CH_heuristica3)
print("Costo incial: ", total_cost3)

solucion_mejorada_all3 = busqueda_local(G, CH_heuristica3, "all")

print("Solucion mejorada con todos: ", solucion_mejorada_all3[0])
print("Costo total: ", solucion_mejorada_all3[1])

### Extra

#### Heuristicas que comienzan desde un nodo random

In [ ]:
def heurisitica1_aleatoria(G):
    
    # Inicializo la solución desde un nodo random
    start_node = random.randrange(0,len(G.nodes)-1,1) #O(1)
    print(start_node)
    visited = [start_node] #O(1)
    current_node = start_node #O(1)

    # Mientras no haya visitado todos los nodos
    while len(visited) < len(G.nodes): #O(n)
        next_node = min(
            (node for node in G.neighbors(current_node) if node not in visited),
            key=lambda node: G[current_node][node]['weight'],
            default=None
        ) #O(n), como es completo, entonces en cada iteración se recorren todos los nodos
        if next_node is None: #O(1)
            break
        
        visited.append(next_node) #O(1)
        current_node = next_node #O(1)
    
    # Calculo el costo totan de la solución
    total_cost = calcular_costo(G, visited) #O(n) recorre al menos una vez cada nodo
    
    return visited, total_cost

# Complejidad total: O(n^2)

In [ ]:
def heuristica2_aleatoria(G):
    # n = |G|
    
    # Inicializo la solución
    start_node = random.randrange(0,len(G.nodes)-1,1) #O(1)
    visited = [start_node]
    current_node = start_node
    print(start_node)
    
    # Mientras no haya visitado todos los nodos
    while len(visited) < len(G.nodes): #O(n)
        neighbors = set(G.neighbors(current_node)) - set(visited) #O(n), porque tengo que recorrer un set de n elementos
        
        best_average = float('inf') #O(1)
        next_node = None #O(1) 
        
        for node in neighbors: #O(n)
            if node not in visited: #O(1), porque es una lista
                average = distancia_promedio_de_vecinos(G, node, visited) #O(n)
                if average < best_average: #O(1)
                    best_average = average #O(1)
                    next_node = node #O(1) 
        
        visited.append(next_node) #O(1)
        current_node = next_node #O(1) 
    
    # Calculo el costo total de la solución
    total_cost = calcular_costo(G, visited) #O(n)
    
    return visited,total_cost

# Complejidad total: O(n^3)


In [ ]:
def heuristica3_aleatoria(G):
    
    start_node = random.randrange(0,len(G.nodes)-1,1)
    visited = [start_node]
    current_node = start_node

    while len(visited) < len(G.nodes):
        next_node = min(
            (node for node in G.neighbors(current_node) if node not in visited),
            key=lambda node: G[node][current_node]['weight'],
            default=None
        )
        if next_node is None:
            break
        
        visited.append(next_node)
        current_node = next_node
    
    total_cost = calcular_costo(G, visited)
    
    return visited, total_cost


##### Exprimentación para heuristicas random

In [ ]:
CH_heuristica1,total_cost = heuristica_1(G)

print("Costo total 1: ", total_cost)

CH_heuristica1_random,total_cost_random = heurisitica1_aleatoria(G)

print("Costo total random: ", total_cost_random)

In [ ]:
# Heuristica 1

print("Solucion inicial: ", CH_heuristica1)
print("Costo incial: ", total_cost)

solucion_mejorada_all = busqueda_local(G, CH_heuristica1, "all")

print("Solucion mejorada con todos: ", solucion_mejorada_all[0])
print("Costo total: ", solucion_mejorada_all[1])

In [ ]:
# Heuristica 1 random

print("Solucion inicial: ", CH_heuristica1_random)
print("Costo incial: ", total_cost_random)

solucion_mejorada_all_random = busqueda_local(G, CH_heuristica1_random, "all")

print("Solucion mejorada con todos: ", solucion_mejorada_all_random[0])
print("Costo total: ", solucion_mejorada_all_random[1])

In [ ]:
CH_heuristica2,total_cost2 = heuristica_2(G)

print("Costo total 2: ", total_cost2)

CH_heuristica2_random,total_cost_random2 = heuristica2_aleatoria(G)

print("Costo total random: ", total_cost_random2)

In [ ]:
# Heuristica 2

print("Solucion inicial: ", CH_heuristica2)
print("Costo incial: ", total_cost2)

solucion_mejorada_all2 = busqueda_local(G, CH_heuristica2, "all")

print("Solucion mejorada con todos: ", solucion_mejorada_all2[0])
print("Costo total: ", solucion_mejorada_all2[1])

In [ ]:
# Heuristica 2 random

print("Solucion inicial: ", CH_heuristica2_random)
print("Costo incial: ", total_cost_random2)

solucion_mejorada_all_random2 = busqueda_local(G, CH_heuristica2_random, "all")

print("Solucion mejorada con todos: ", solucion_mejorada_all_random2[0])
print("Costo total: ", solucion_mejorada_all_random2[1])

In [ ]:
CH_heuristica3,total_cost3 = heuristica_3(G)

print("Costo total 3: ", total_cost2)

CH_heuristica3_random,total_cost_random3 = heuristica3_aleatoria(G)

print("Costo total random: ", total_cost_random3)

In [ ]:
# Heuristica 2

print("Solucion inicial: ", CH_heuristica3)
print("Costo incial: ", total_cost3)

solucion_mejorada_all3 = busqueda_local(G, CH_heuristica3, "all")

print("Solucion mejorada con todos: ", solucion_mejorada_all3[0])
print("Costo total: ", solucion_mejorada_all3[1])

In [ ]:
# Heuristica 2 random

print("Solucion inicial: ", CH_heuristica3_random)
print("Costo incial: ", total_cost_random3)

solucion_mejorada_all_random3 = busqueda_local(G, CH_heuristica3_random, "all")

print("Solucion mejorada con todos: ", solucion_mejorada_all_random3[0])
print("Costo total: ", solucion_mejorada_all_random3[1])